In [ ]:
# The code was removed by Watson Studio for sharing.

# Scraping and Preparing Dataframe of Toronto neighbourhoods

### Downloading all required Packages.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#!pip install -U scikit-learn scipy matplotlib
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests  #to handle web requests

print('Libraries imported.')

Libraries imported.


### This is link to Wikipedia Page for Canada we will be using as base for data scraping.

In [22]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### We will use BeautifuSoup package for Data Scraping

In [44]:
#!pip install BeautifulSoup4
from bs4 import BeautifulSoup

soup = BeautifulSoup(website_url,'html.parser')
# This will be a BIG soup !

### We will use body of table to store in Pandas Dataframe.

In [46]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

### Forming DataFrame with required values

In [47]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()

,,Postal Code,,Borough,,Neighborhood,
1,,M1A,,Not assigned,,,
2,,M2A,,Not assigned,,,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


### Droping rows with 'Not assigned' values in Borough

In [54]:
df4 = df3[df3.Borough != 'Not assigned']
df4.head()

,,Postal Code,,Borough,,Neighborhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


### Combining neighborhoods with same postal code in one cell 

In [58]:
df5 = df4.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df5.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Assigning Value of Borough to Neighborhoods in case Neighborhoods has no value assigned.

In [61]:
df5.loc[df5['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df5['Borough']
df5.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Shape of the Dataframe

In [63]:
Toronto_DF = df5
Toronto_DF.shape

(103, 3)

### This is final prepared Dataframe for Toronto.

In [66]:
Toronto_DF.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## Thank you.

### I am going to store this DataFrame as .CSV into IBM cloud so i can use it later in second part of this assignment.

In [101]:
from project_lib import Project
project = Project(None,"12c46224-03a3-4888-8f15-59f253be0f49", "p-724875fe3ace0613160dc93a797d833ab9899d9a")
pc = project.project_context

project.save_data(file_name = "Toronto_DF.csv",data = Toronto_DF.to_csv(index=False))

{'file_name': 'Toronto_DF.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstoneproject-donotdelete-pr-wlmmgfrwp1yq58',
 'asset_id': '03bc2a73-03d8-461b-86ae-3e16e7f8b2c6'}